In [1]:

import logging
import multiprocessing
import os
from collections import Counter
from typing import List, Union

from gensim.models import FastText
from indicnlp.tokenize.indic_tokenize import trivial_tokenize_indic
from tqdm import tqdm

tqdm.pandas()

In [2]:
# set up the logging to monitor gensim
logging.basicConfig(
    format="%(levelname)s - %(asctime)s: %(message)s",
    datefmt='%H:%M:%S',
    level=logging.INFO,
)

logger = logging.getLogger(__name__)

## Utilities

In [3]:
def tokenize_text(text: List[str]) -> List[List[str]]:
    """Tokenize text"""
    return [trivial_tokenize_indic(sent) for sent in tqdm(text, desc='tokenize', unit=' sentences')]

In [4]:
def train_fasttext(tokenized_text: List[List[str]], size: int = 100, window: int = 5, min_count: int = 1,
                   epochs: int = 10,
                   random_seed: int = 123, vec_file_path: Union[str, None] = None, ):
    """Learn fasttext embeddings"""
    # count the number of cores
    cores = multiprocessing.cpu_count()
    # create fasttext model
    model = FastText(
        size=size,
        window=window,
        min_count=min_count,
        workers=cores - 1,
        seed=random_seed,
    )
    # build vocab
    model.build_vocab(sentences=tokenized_text, progress_per=1000000)  # show progress after processing every 1M words
    # train
    model.train(sentences=tokenized_text, total_examples=model.corpus_count, epochs=epochs,
                report_delay=10)  # show progress after every 10 seconds
    if vec_file_path is not None:
        model.save(vec_file_path)
    return model

## Load data
For learning the Odia word embeddings, we need monolingual Odia text data. You can possibly scrape data from an online source such as Wikipedia. For our experiments now, let's take the Odia monolingual text data available as part of the Indic NLP corpus.

In [5]:
filename = os.path.join('data/or')
assert os.path.isfile(filename)  # sanity check

In [6]:
with open(filename, 'r', encoding='utf-8') as f:
    lines = [s.strip() for s in tqdm(f.readlines(), desc='read lines from file')]

read lines from file: 100%|██████████| 3594672/3594672 [00:02<00:00, 1405592.12it/s]


In [7]:
# tokenize
tokens = tokenize_text(lines)

tokenize: 100%|██████████| 3594672/3594672 [01:33<00:00, 38276.08 sentences/s]


In [8]:
num_running_toks, num_unique_toks = 0, 0
counter = Counter()
for toks in tqdm(tokens, desc='compute frequencies of tokens', unit=' sentences'):
    counter.update(toks)

compute frequencies of tokens: 100%|██████████| 3594672/3594672 [00:26<00:00, 135718.34 sentences/s]


In [9]:
print(f'Number of sentences: {len(lines):,}')
print(f'Number of unique words or equivalantly, the size of vocabulary: {len(counter):,}')
print(f'Number of running words: {sum([freq for _, freq in counter.items()]):,}')

Number of sentences: 3,594,672
Number of unique words or equivalantly, the size of vocabulary: 778,862
Number of running words: 51,151,273


In [10]:

# most common words
counter.most_common(n=20)

[('।', 3393061),
 (',', 1191253),
 ('ଓ', 534792),
 ('ଏହି', 437185),
 ('ପାଇଁ', 373726),
 ('ସେ', 240775),
 ('ବୋଲି', 239837),
 ('ପରେ', 224959),
 ('କରି', 221628),
 ('ଏକ', 213516),
 ('ମଧ୍ୟ', 210907),
 ('ଏବଂ', 198988),
 ('କରିଥିଲେ', 195168),
 ('ସହ', 177040),
 ('-', 174796),
 ('ଖବର', 169373),
 ('.', 166728),
 ('କରିବା', 166276),
 ('ନେଇ', 161728),
 ('ବେଳେ', 156327)]

## Learn embeddings

In [11]:
fasttext_model = train_fasttext(tokenized_text=tokens, size=100, window=5, min_count=20, epochs=10, random_seed=123,
                                vec_file_path=os.path.join('embeddings.txt'))

INFO - 15:36:14: resetting layer weights
INFO - 15:36:28: collecting all words and their counts
INFO - 15:36:28: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 15:36:35: PROGRESS: at sentence #1000000, processed 14406915 words, keeping 356423 word types
INFO - 15:36:40: PROGRESS: at sentence #2000000, processed 28227997 words, keeping 518060 word types
INFO - 15:36:45: PROGRESS: at sentence #3000000, processed 42532970 words, keeping 692676 word types
INFO - 15:36:48: collected 778862 word types from a corpus of 51151273 raw words and 3594672 sentences
INFO - 15:36:48: Loading a fresh vocabulary
INFO - 15:36:49: effective_min_count=20 retains 72827 unique words (9% of original 778862, drops 706035)
INFO - 15:36:49: effective_min_count=20 leaves 49262024 word corpus (96% of original 51151273, drops 1889249)
INFO - 15:36:49: deleting the raw counts dictionary of 778862 items
INFO - 15:36:49: sample=0.001 downsamples 22 most-common words
INFO - 15:36:49: downsamp

In [12]:
# fasttext model's vocab size
print(f'fasttext model\'s vocabulary size: {len(fasttext_model.wv.vocab):,}')

fasttext model's vocabulary size: 72,827


## Evaluate embeddings
Here we evaluate the embeddings learned by just 👀 at the neighbours of a few words and examining if they are similar.



In [13]:
## find words similar to a given word
fasttext_model.wv.most_similar('ଗଛ', topn=10)  # TODO: misspell it

INFO - 16:05:10: precomputing L2-norms of word weight vectors
INFO - 16:05:10: precomputing L2-norms of ngram weight vectors


[('ଗଛଡାଳ', 0.8653923273086548),
 ('ବରଗଛ', 0.8522686958312988),
 ('ଗଛର', 0.8209975957870483),
 ('ଗଛଟି', 0.801505982875824),
 ('ଆମ୍ବଗଛ', 0.7687293291091919),
 ('ଗଛଟିଏ', 0.7585610747337341),
 ('କଦଳୀଗଛ', 0.7574436664581299),
 ('ଫୁଲଗଛ', 0.753203809261322),
 ('ଗଛକାଟି', 0.7440799474716187),
 ('ଶାଳଗଛ', 0.7383095622062683)]

In [14]:
fasttext_model.wv.most_similar('ସଙ୍ଗୀତ', topn=10)

[('ସଂଗୀତ', 0.9609819054603577),
 ('ସଂଙ୍ଗୀତ', 0.9506467580795288),
 ('ସଙ୍ଗୀତର', 0.9171082973480225),
 ('ସଙ୍ଗୀତକାର', 0.893383264541626),
 ('ସଂଗୀତକାର', 0.8804852366447449),
 ('ସଂଗୀତର', 0.8760529160499573),
 ('ସଙ୍ଗୀତଜ୍ଞ', 0.8728147745132446),
 ('ଗୀତିନାଟ୍ୟ', 0.8677797913551331),
 ('ନୃତ୍ୟଗୀତ', 0.8639868497848511),
 ('ସଂଗୀତଜ୍ଞ', 0.8267605304718018)]

In [15]:
fasttext_model.wv.most_similar('ଚଳଚ୍ଚିତ୍ର', topn=10)

[('ଚଳଚିତ୍ର', 0.930408239364624),
 ('ଚଳଚ୍ଚିତ୍ରଟି', 0.9154518246650696),
 ('ଚଳଚ୍ଚିତ୍ରଟିର', 0.9073564410209656),
 ('ଚଳଚ୍ଚିତ୍ରର', 0.8907100558280945),
 ('ସିନେମା', 0.833586573600769),
 ('ସିନେମାଟୋଗ୍ରାଫର', 0.8292860388755798),
 ('ଫିଲ୍ମ', 0.826623797416687),
 ('ଚଳଚ୍ଚିତ୍ରରେ', 0.8206254243850708),
 ('ଚଳଚ୍ଚିତ୍ରଟିରେ', 0.8183248043060303),
 ('ଚଳଚ୍ଚିତ୍ରକୁ', 0.8024610280990601)]

**Try some misspelled words**

In [16]:
fasttext_model.wv.most_similar('ଚଳଚ୍ଚିତ', topn=10)

[('ଚଳଚ୍ଚିତ୍ର', 0.8048502206802368),
 ('ଚଳଚ୍ଚିତ୍ରଟିର', 0.7769005298614502),
 ('ଚଳଚ୍ଚିତ୍ରଟି', 0.7636350393295288),
 ('ସମୁଚ୍ଚିତ', 0.7427504658699036),
 ('ଚଳଚ୍ଚିତ୍ରର', 0.7388564348220825),
 ('ଲୋକପ୍ରିୟ', 0.7327723503112793),
 ('ସନ୍ନିବେଶିତ', 0.7062867879867554),
 ('ଫିଲ୍ମ', 0.7052443027496338),
 ('ଚଳଚ୍ଚିତ୍ରଟିରେ', 0.7045912742614746),
 ('ସିନେମାଟୋଗ୍ରାଫର', 0.6947799921035767)]

In [17]:
fasttext_model.wv.most_similar('ସଗୀତ', topn=10)

[('ଗୀତ', 0.9333454370498657),
 ('ସଂଙ୍ଗୀତ', 0.8912193179130554),
 ('ନୃତ୍ୟଗୀତ', 0.8894705772399902),
 ('ନାଚଗୀତ', 0.8886850476264954),
 ('ସଂଗୀତ', 0.8765240907669067),
 ('ସଙ୍ଗୀତ', 0.8741494417190552),
 ('ଗୀତ୍', 0.8588917255401611),
 ('ଗୀତିନାଟ୍ୟ', 0.8219820261001587),
 ('ଗୀତର', 0.7932537794113159),
 ('ସଙ୍ଗୀତର', 0.7914323806762695)]